In [1]:
# Replicate ITI results, make sure ITI utils and probing utils work right

#%%
from IPython import get_ipython

ipython = get_ipython()
# Code to automatically update the TransformerLens code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")
    
import plotly.io as pio
# pio.renderers.default = "png"
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

from tqdm import tqdm
# from utils.probing_utils import ModelActs
from utils.dataset_utils import CounterFact_Dataset, TQA_MC_Dataset, EZ_Dataset

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

from utils.iti_utils import patch_iti

from utils.analytics_utils import plot_probe_accuracies, plot_norm_diffs, plot_cosine_sims

/tmp/ipykernel_5452/4073292201.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_5452/4073292201.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:

device = "cuda"
print("loading model")
model = HookedTransformer.from_pretrained(
    "gpt2-xl",
    center_unembed=False,
    center_writing_weights=False,
    fold_ln=False,
    refactor_factored_attn_matrices=True,
    device=device,
)
# model.to(device)
print("done")
model.cfg.total_heads = model.cfg.n_heads * model.cfg.n_layers

model.reset_hooks()

loading model


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
done


In [3]:
from utils.dataset_utils import MS_Dataset, Elem_Dataset, MisCons_Dataset, Kinder_Dataset, HS_Dataset, BoolQ_Question_Dataset, TruthfulQA_Tfn, CounterFact_Tfn, Fever_Tfn, BoolQ_Tfn, Creak_Tfn, CommonClaim_Tfn
random_seed = 5

datanames = ["MS", "Elem", "MisCons", "TruthfulQA"]

ms_data = MS_Dataset(model.tokenizer, questions=True)
elem_data = Elem_Dataset(model.tokenizer, questions=True)
miscons_data = MisCons_Dataset(model.tokenizer, questions=True)
tqa_data = TruthfulQA_Tfn(model.tokenizer, questions=True)

datasets = {"MS": ms_data, "Elem": elem_data, "MisCons": miscons_data, "TruthfulQA": tqa_data}

Found cached dataset parquet (/root/.cache/huggingface/datasets/notrichardren___parquet/notrichardren--elem_tf-64ec49cd4cd5be64/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/root/.cache/huggingface/datasets/notrichardren___parquet/notrichardren--ms_tf-728c6138d8f6c1c5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/root/.cache/huggingface/datasets/notrichardren___parquet/notrichardren--misconceptions_tf-131f43b181040ffa/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/root/.cache/huggingface/datasets/notrichardren___parquet/notrichardren--truthfulness-4380c84abeab6c8f/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
from utils.new_probing_utils import SmallModelActs
n_acts = 50
acts = {}

for name in datanames[:1]:
    # acts[name] = ModelActs(model, datasets[name], act_types=["z", "mlp_out", "resid_post", "resid_pre", "logits"])
    acts[name] = SmallModelActs(model, datasets[name], act_types=["z", "logits"])
    model_acts: SmallModelActs = acts[name]
    model_acts.gen_acts(N=n_acts, id=f"{name}_gpt2xl_{n_acts}")
    # model_acts.load_acts(id=f"{name}_gpt2xl_{n_acts}", load_probes=False)
    print("training probes now")
    model_acts.train_probes("z", max_iter=1000)

100%|██████████| 50/50 [00:06<00:00,  7.16it/s]


Finished generating activations
Activations stacked


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   # acts[name] = ModelActs(model, datasets[name], act_types=["z", "mlp_out", "resid_po    │
│    7 │   acts[name] = SmallModelActs(model, datasets[name], act_types=["z", "logits"])           │
│    8 │   model_acts: SmallModelActs = acts[name]                                                 │
│ ❱  9 │   model_acts.gen_acts(N=n_acts, id=f"{name}_gpt2xl_{n_acts}")                             │
│   10 │   # model_acts.load_acts(id=f"{name}_gpt2xl_{n_acts}", load_probes=False)                 │
│   11 │   print("training probes now")                                                            │
│   12 │   model_acts.train_probes("z", max_iter=1000)                                             │
│                                                                                                  │
│ /home/phil/deep_learning/iti_capstone/utils/new_probing_utils.py:337 in gen_acts                 │
│                                                                                                  │
│   334 │   │   │   torch.save(self.data_indices, f'{filepath}{id}_data_indices.pt')               │
│   335 │   │   │   for act_type in self.act_types:                                                │
│   336 │   │   │   │   with open(f'{filepath}{id}_{act_type}_acts.pt', 'wb') as f:                │
│ ❱ 337 │   │   │   │   │   pickle.dump(self.activations[act_type], f)                             │
│   338 │   │   │   print(f"Stored at {id}")                                                       │
│   339                                                                                            │
│   340                                                                                            │
│                                                                                                  │
│ /home/phil/.conda/envs/iti-cap/lib/python3.8/site-packages/torch/storage.py:770 in __reduce__    │
│                                                                                                  │
│    767 │                                                                                         │
│    768 │   def __reduce__(self):                                                                 │
│    769 │   │   b = io.BytesIO()                                                                  │
│ ❱  770 │   │   torch.save(self, b, _use_new_zipfile_serialization=False)                         │
│    771 │   │   return (_load_from_bytes, (b.getvalue(),))                                        │
│    772 │                                                                                         │
│    773 │   def data_ptr(self):                                                                   │
│                                                                                                  │
│ /home/phil/.conda/envs/iti-cap/lib/python3.8/site-packages/torch/serialization.py:445 in save    │
│                                                                                                  │
│    442 │   │   │   return                                                                        │
│    443 │   else:                                                                                 │
│    444 │   │   with _open_file_like(f, 'wb') as opened_file:                                     │
│ ❱  445 │   │   │   _legacy_save(obj, opened_file, pickle_module, pickle_protocol)                │
│    446                                                                                           │
│    447                                                                                           │
│    448 def _legacy_save(obj, f, pickle_module, pickle_proto

In [ ]:
acts["MS"].activations

In [ ]:
acts["MS"].train_probes("z", max_iter=1000)